In [3]:
import numpy as np

In [4]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("key/bluebookkey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://bluebookai2020.firebaseio.com"
})

In [5]:
from firebase_admin import db

In [6]:
import pandas as pd

### Realtime Database Code

In [7]:
test1_merge=pd.read_csv("test1_merge.csv").drop("Unnamed: 0", axis=1)

In [8]:
test1_merge.head()

,test_num,testDate,userID,total,first,last,1_01,1_02,1_03,1_04,...,4_29,4_30,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38
0,1,2017-07-11,70,1300,Jimin,Lee,A,C,A,D,...,B,C,0.8,4,0.96,4.5,.16,32.0,371.0,25
1,1,2016-12-17,169,1200,Jeannie,Chung,A,C,A,D,...,B,C,0.8,8,NaN,4.3,.88,NaN,507.0,NaN
2,1,2018-09-11,309,1040,John,Choi,D,C,A,D,...,C,B,0.8,NaN,NaN,NaN,NaN,8.0,NaN,NaN
3,1,2017-06-10,380,640,Seungpyo,Cho,NaN,NaN,NaN,NaN,...,C,C,0.8,8,1.42,9.2,0.6,12.0,161.0,5
4,1,2018-08-28,453,1320,Hyunseung,Lee,A,A,A,D,...,C,C,0.8,8,1.92,4.5,.84,12.0,371.0,25


In [191]:
# must be all string !!
test1_merge = test1_merge.astype(str)
test1_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Columns: 160 entries, test_num to 4_38
dtypes: object(160)
memory usage: 78.8+ KB


In [123]:
# change it all str/ no needed to be filled in 
#test1_merge.fillna('nan', inplace=True)
#null -> change it as nan string 

In [13]:
def dict_creator(df):
    tmp = []
    test_col = df.columns
    for i in range(df.shape[0]):
        tmp.append(dict(zip(test_col, df.iloc[i].tolist())))
    
    tmp2 = []
    userID = df['userID'].values
    for i in range(df.shape[0]):
        tmp2.append({f"{userID[i]}":tmp[i]})
    dict_df = tmp2
    return dict_df

In [14]:
# all update all dataset
def sat_upload_to_firebase(df, test_num):
    test_num = str(test_num)
    path = 'sat/' + test_num + "/"
    for num in range(len(df)):
        ref = db.reference(path)
        ref.update(df[num])

In [195]:
#  convert it into dict json
dict_test1=dict_creator(test1_merge)

In [196]:
# upload test1 dataset into firebase using def 
sat_upload_to_firebase(dict_test1, "test1")

In [161]:
# firebase initialize database setup
ref = db.reference('sat/test1/')
ref.set({"": ""})

In [10]:
# upload all sat dataset into firebase
standard_col=pd.read_csv("standard_sat_data_col.csv", index_col=0)['0'].values
df_test=pd.read_csv("sat_standard_dataset_rawdata.csv")
print(df_test.shape)
df_test[standard_col].head()

(853, 160)


,test_num,testDate,userID,first,last,1_01,1_02,1_03,1_04,1_05,...,4_29,4_30,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38
0,R1,07/26/2019,9999,Test,Level,C,B,D,A,C,...,C,D,9,'31/9,124,3,4,2.6,3.,8
1,R1,05/01/2019,9212,Jang,Daniel,C,C,D,D,D,...,C,C,.8,NaN,NaN,NaN,NaN,8,371,800
2,R1,01/26/2019,387,Pak,Richard,C,A,D,A,C,...,B,D,10,'31/4,98,5,1.25,26,30,8
3,R1,01/26/2019,575,Kim,Alexis,C,B,D,A,C,...,B,NaN,10,31,100,5,'5/4,2.6,30,8
4,R1,01/26/2019,722,Taktak,Begum,C,B,D,A,C,...,B,D,NaN,31,98,5,'5/4,2,30,8


In [24]:
# checking !!!!  ~~~ retrive dataset from firebase 
ref = db.reference("sat/rawdata/")
sat_test = ref.get()
pd.DataFrame(sat_test).T[standard_col].reset_index(drop="index").head()

,test_num,testDate,userID,first,last,1_01,1_02,1_03,1_04,1_05,...,4_29,4_30,4_31,4_32,4_33,4_34,4_35,4_36,4_37,4_38
0,R5,09/30/2019,4,Lee,Gloria,B,A,A,B,D,...,D,A,195,.4,30,.28,3,6,.48,.1
1,23,10/05/2019,8,Lee,Erin,D,C,C,A,C,...,A,C,6.12,200,5,6.48,44,650,4,14
2,6,08/31/2016,9,kim,Junesoo,A,C,D,A,C,...,B,nan,.24,4,25,.36,.49,5,1.14,1112
3,9,03/20/2017,23,Kim,Jian,D,C,D,B,C,...,B,A,550,15,24,8192,5800,3,3.25,nan
4,5,07/28/2016,33,Chae,Brian,A,B,A,D,A,...,D,A,2.5,9,36,800,3.75,29,3,992


In [56]:
# upload all sat dataset into firebase
standard_col=pd.read_csv("standard_sat_data_col.csv", index_col=0)['0'].values
df_test=pd.read_csv("sat_standard_dataset_rawdata.csv").drop("Unnamed: 0", axis=1)
print(df_test.shape)

# upload raw dataset into firebase 
df = df_test.astype(str)
#df=dict_creator(df_test)

tmp = []
test_col = df.columns
for i in range(df.shape[0]):
    tmp.append(dict(zip(test_col, df.iloc[i].tolist())))

dict_df=tmp
    
for num in range(len(dict_df)):
        ref = db.reference("sat/raw/")
        ref.push().update(dict_df[num])


(853, 159)


In [33]:
# upload raw dataset ans into firebase 

raw_ans=pd.read_csv("rawdata_ans/sat_rawdata_ans.csv")

df = raw_ans.astype(str)
tmp = []
test_col = df.columns
for i in range(df.shape[0]):
    tmp.append(dict(zip(test_col, df.iloc[i].tolist())))

tmp2 = []
test_num = df['test_num'].values
for i in range(df.shape[0]):
    tmp2.append({f"{test_num[i]}":tmp[i]})
dict_df = tmp2

for num in range(len(dict_df)):
        ref = db.reference("sat/raw_ans/")
        ref.update(dict_df[num])

In [49]:
# upload raw dataset total score into firebase 

raw_score=pd.read_csv("rawdata_ans/sat_total_score.csv")
df = raw_score.astype(str)


tmp = []
test_col = df.columns
for i in range(df.shape[0]):
    tmp.append(dict(zip(test_col, df.iloc[i].tolist())))

dict_df =tmp
    
for num in range(len(dict_df)):
        ref = db.reference("sat/raw_total_score/")
        ref.push().update(dict_df[num])